In [1]:
from collections import defaultdict
import numpy as np
import pandas as pd
import csv

models = ['moderate','weak','strong']

Nbootstrap = 10
num_random_contexts = 1000


## CR9114

In [28]:
#### antibody choice ####
antibody = "9114"
#antibody = "6261" 

antigens = ["H1", "H3", "FluB"] 
L = 16



In [51]:
# read likelihoods from files and make dictionary
prob_dict = {}
for model in models:    
    with open('data/likelihoods_'+antibody+'_'+model+'.csv','r') as readfile:
        proba_reader = csv.reader(readfile)
        for row in proba_reader:
            key = tuple(row[:L])
            if 'mixed' in key:
                key = tuple(['average']*L)
            values = [float(x) for x in row[L:]]
            if model == 'strong':
                if 'average' in key:
                    mean = np.nan
                    sem = np.nan
                else:
                    mean = np.mean(values)
                    sem = np.std(values,ddof=1)
            else:
                mean = np.log(np.mean(values))
                sem = np.std(values,ddof=1)/np.mean(values)

            if key in prob_dict.keys():
                prob_dict[key].append(mean)
                prob_dict[key].append(sem)
            else:
                if key == tuple(['H1']*L): scenario_type = 'H1_only'
                elif key == tuple(['H3']*L): scenario_type = 'H3_only'
                elif key == tuple(['FluB']*L): scenario_type = 'FluB_only'
                elif 'average' in key: scenario_type = 'mixed_average'
                else: scenario_type = 'sequential'
                prob_dict[key] = [scenario_type,mean,sem]
                
        readfile.close()
        
    with open('data/likelihoods_'+antibody+'_'+model+'_random.csv','r') as readfile:  
        
        random_values = np.empty(num_random_contexts)
        random_errs = np.empty(num_random_contexts)
        
        proba_reader = csv.reader(readfile)
        j = 0
        
        for row in proba_reader:
            values = [float(x) for x in row[L:]]
            if model == 'strong':
                random_values[j] = np.mean(values)
                random_errs[j] = np.std(values,ddof=1)          
            else:
                random_values[j] = np.log(np.mean(values))
                random_errs[j] = np.std(values,ddof=1)/np.mean(values)
            j += 1
        readfile.close()

    mean = np.mean(random_values)
    sem = np.std(random_errs,ddof=1)
    
    key = tuple(['random']*L)
    if key in prob_dict:
        prob_dict[key].append(mean)
        prob_dict[key].append(sem)
    else:
        prob_dict[key] = ['mixed_random',mean,sem]
                
print(len(prob_dict))


725


In [52]:
# reformat as dataframe
prob_df = pd.DataFrame({'ScenarioType':[x[0] for x in prob_dict.values()],
    'Ag1':[x[0] for x in prob_dict.keys()],
    'Ag2':[x[1] for x in prob_dict.keys()],
    'Ag3':[x[2] for x in prob_dict.keys()],
    'Ag4':[x[3] for x in prob_dict.keys()],
    'Ag5':[x[4] for x in prob_dict.keys()],
    'Ag6':[x[5] for x in prob_dict.keys()],
    'Ag7':[x[6] for x in prob_dict.keys()],
    'Ag8':[x[7] for x in prob_dict.keys()],
    'Ag9':[x[8] for x in prob_dict.keys()],
    'Ag10':[x[9] for x in prob_dict.keys()],
    'Ag11':[x[10] for x in prob_dict.keys()],
    'Ag12':[x[11] for x in prob_dict.keys()],
    'Ag13':[x[12] for x in prob_dict.keys()],
    'Ag14':[x[13] for x in prob_dict.keys()],
    'Ag15':[x[14] for x in prob_dict.keys()],
    'Ag16':[x[15] for x in prob_dict.keys()],
    'MeanLogProb_Moderate':[x[1] for x in prob_dict.values()],
    'StdErrLogProb_Moderate':[x[2] for x in prob_dict.values()], 
    'MeanLogProb_Weak':[x[3] for x in prob_dict.values()],
    'StdErrLogProb_Weak':[x[4] for x in prob_dict.values()], 
    'MeanPaths_Strong':[x[5] for x in prob_dict.values()],
    'StdErrPaths_Strong':[x[6] for x in prob_dict.values()] 

})

prob_df = prob_df.sort_values(by=['MeanLogProb_Moderate'],ascending=False)

prob_df.to_csv('data/9114_scenario_probabilities.csv',index=False)

prob_df

ScenarioType   Ag1   Ag2   Ag3   Ag4   Ag5   Ag6   Ag7   Ag8   Ag9  ...  \
437   sequential    H1    H1    H1    H1    H1    H3    H3    H3    H3  ...   
554   sequential    H1    H1    H1    H1    H1    H1    H3    H3    H3  ...   
476   sequential    H1    H1    H1    H1    H1    H3    H3    H3    H3  ...   
205   sequential    H1    H1    H1    H1    H1    H1    H3    H3    H3  ...   
716   sequential    H1    H1    H1    H1    H3    H3    H3    H3    H3  ...   
..           ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   
512   sequential    H3    H3    H3    H3  FluB  FluB  FluB  FluB  FluB  ...   
192    FluB_only  FluB  FluB  FluB  FluB  FluB  FluB  FluB  FluB  FluB  ...   
280   sequential    H3    H3  FluB  FluB  FluB  FluB  FluB  FluB  FluB  ...   
83    sequential    H3    H3    H3  FluB  FluB  FluB  FluB  FluB  FluB  ...   
265   sequential    H3  FluB  FluB  FluB  FluB  FluB  FluB  FluB  FluB  ...   

     Ag13  Ag14  Ag15  Ag16 MeanLogProb_Moderate StdErrLogProb_Moderate  \
437    H3  FluB  FluB  FluB           -12.456035               1.037490   
554    H3  FluB  FluB  FluB           -12.663612               1.145840   
476  FluB  FluB  FluB  FluB           -12.861887               0.777849   
205  FluB  FluB  FluB  FluB           -13.235156               0.947050   
716    H3  FluB  FluB  FluB           -13.425262               0.999357   
..    ...   ...   ...   ...                  ...                    ...   
512  FluB  FluB  FluB  FluB           -62.866281               2.301367   
192  FluB  FluB  FluB  FluB           -62.866281               2.301367   
280  FluB  FluB  FluB  FluB           -62.866281               2.301367   
83   FluB  FluB  FluB  FluB           -62.866281               2.301367   
265  FluB  FluB  FluB  FluB           -62.866281               2.301367   

    MeanLogProb_Weak  StdErrLogProb_Weak  MeanPaths_Strong  StdErrPaths_Strong  
437       -10.180258            0.273735        12021371.3        9.013946e+06  
554       -10.163164            0.279486        21052293.6        1.470031e+07  
476       -10.622306            0.272821         3947722.4        2.314285e+06  
205       -10.628184            0.278974         6170172.1        3.376908e+06  
716       -10.284436            0.267540         3652536.5        2.941756e+06  
..               ...                 ...               ...                 ...  
512       -15.257277            0.104440               0.0        0.000000e+00  
192       -15.257277            0.104440               0.0        0.000000e+00  
280       -15.257277            0.104440               0.0        0.000000e+00  
83        -15.257277            0.104440               0.0        0.000000e+00  
265       -15.257277            0.104440               0.0        0.000000e+00  

[725 rows x 23 columns]

## CR6261 

In [53]:
#### 6261 ####
antibody = "6261"

antigens = ["H1", "H9"]
L = 11



In [54]:
# read likelihoods from files and make dictionary
prob_dict = {}
for model in models:    
    with open('data/likelihoods_'+antibody+'_'+model+'.csv','r') as readfile:
        proba_reader = csv.reader(readfile)
        for row in proba_reader:
            key = tuple(row[:L])
            if 'mixed' in key:
                key = tuple(['average']*L)
            values = [float(x) for x in row[L:]]
            if model == 'strong':
                if 'average' in key:
                    mean = np.nan
                    sem = np.nan
                else:
                    mean = np.mean(values)
                    sem = np.std(values,ddof=1)
            else:
                mean = np.log(np.mean(values))
                sem = np.std(values,ddof=1)/np.mean(values)

            if key in prob_dict.keys():
                prob_dict[key].append(mean)
                prob_dict[key].append(sem)
            else:
                if key == tuple(['H1']*L): scenario_type = 'H1_only'
                elif key == tuple(['H9']*L): scenario_type = 'H9_only'
                elif 'average' in key: scenario_type = 'mixed_average'
                else: scenario_type = 'sequential'
                prob_dict[key] = [scenario_type,mean,sem]
                
        readfile.close()
        
    with open('data/likelihoods_'+antibody+'_'+model+'_random.csv','r') as readfile:  
        
        random_values = np.empty(num_random_contexts)
        random_errs = np.empty(num_random_contexts)
        
        proba_reader = csv.reader(readfile)
        j = 0
        
        for row in proba_reader:
            values = [float(x) for x in row[L:]]
            if model == 'strong':
                random_values[j] = np.mean(values)
                random_errs[j] = np.std(values,ddof=1)          
            else:
                random_values[j] = np.log(np.mean(values))
                random_errs[j] = np.std(values,ddof=1)/np.mean(values)
            j += 1
        readfile.close()

    mean = np.mean(random_values)
    sem = np.std(random_errs,ddof=1)
    
    key = tuple(['random']*L)
    if key in prob_dict:
        prob_dict[key].append(mean)
        prob_dict[key].append(sem)
    else:
        prob_dict[key] = ['mixed_random',mean,sem]
                
print(len(prob_dict))

24


In [55]:
# reformat as dataframe
prob_df = pd.DataFrame({'ScenarioType':[x[0] for x in prob_dict.values()],
    'Ag1':[x[0] for x in prob_dict.keys()],
    'Ag2':[x[1] for x in prob_dict.keys()],
    'Ag3':[x[2] for x in prob_dict.keys()],
    'Ag4':[x[3] for x in prob_dict.keys()],
    'Ag5':[x[4] for x in prob_dict.keys()],
    'Ag6':[x[5] for x in prob_dict.keys()],
    'Ag7':[x[6] for x in prob_dict.keys()],
    'Ag8':[x[7] for x in prob_dict.keys()],
    'Ag9':[x[8] for x in prob_dict.keys()],
    'Ag10':[x[9] for x in prob_dict.keys()],
    'Ag11':[x[10] for x in prob_dict.keys()],
    'MeanLogProb_Moderate':[x[1] for x in prob_dict.values()],
    'StdErrLogProb_Moderate':[x[2] for x in prob_dict.values()], 
    'MeanLogProb_Weak':[x[3] for x in prob_dict.values()],
    'StdErrLogProb_Weak':[x[4] for x in prob_dict.values()], 
    'MeanPaths_Strong':[x[5] for x in prob_dict.values()],
    'StdErrPaths_Strong':[x[6] for x in prob_dict.values()] 

})

prob_df = prob_df.sort_values(by=['MeanLogProb_Moderate'],ascending=False)

prob_df.to_csv('data/6261_scenario_probabilities.csv',index=False)

prob_df

ScenarioType      Ag1      Ag2      Ag3      Ag4      Ag5      Ag6  \
15     sequential       H1       H1       H1       H1       H9       H9   
1      sequential       H1       H1       H1       H9       H9       H9   
22     sequential       H1       H1       H1       H1       H1       H9   
21     sequential       H1       H1       H1       H1       H1       H1   
0      sequential       H1       H1       H1       H1       H1       H1   
18     sequential       H1       H1       H1       H1       H1       H1   
13     sequential       H1       H1       H9       H9       H9       H9   
20  mixed_average  average  average  average  average  average  average   
10     sequential       H1       H1       H1       H1       H1       H1   
23   mixed_random   random   random   random   random   random   random   
2      sequential       H1       H1       H1       H1       H1       H1   
3      sequential       H1       H9       H9       H9       H9       H9   
16        H1_only       H1       H1       H1       H1       H1       H1   
5      sequential       H9       H9       H9       H9       H9       H1   
14     sequential       H9       H9       H9       H9       H1       H1   
6      sequential       H9       H9       H9       H9       H9       H9   
4      sequential       H9       H9       H9       H9       H9       H9   
19     sequential       H9       H9       H9       H9       H9       H9   
12     sequential       H9       H9       H9       H9       H9       H9   
7      sequential       H9       H9       H9       H9       H9       H9   
8      sequential       H9       H9       H9       H1       H1       H1   
17        H9_only       H9       H9       H9       H9       H9       H9   
9      sequential       H9       H1       H1       H1       H1       H1   
11     sequential       H9       H9       H1       H1       H1       H1   

        Ag7      Ag8      Ag9     Ag10     Ag11  MeanLogProb_Moderate  \
15       H9       H9       H9       H9       H9             -7.075328   
1        H9       H9       H9       H9       H9             -7.326466   
22       H9       H9       H9       H9       H9             -7.785895   
21       H9       H9       H9       H9       H9             -7.976453   
0        H1       H9       H9       H9       H9             -8.563220   
18       H1       H1       H9       H9       H9             -9.737674   
13       H9       H9       H9       H9       H9             -9.974330   
20  average  average  average  average  average            -10.924602   
10       H1       H1       H1       H9       H9            -11.082049   
23   random   random   random   random   random            -11.637744   
2        H1       H1       H1       H1       H9            -12.240444   
3        H9       H9       H9       H9       H9            -12.695078   
16       H1       H1       H1       H1       H1            -13.091652   
5        H1       H1       H1       H1       H1            -14.161228   
14       H1       H1       H1       H1       H1            -14.704847   
6        H1       H1       H1       H1       H1            -14.748611   
4        H9       H9       H9       H1       H1            -14.802438   
19       H9       H9       H1       H1       H1            -15.131430   
12       H9       H1       H1       H1       H1            -15.179305   
7        H9       H9       H9       H9       H1            -15.254679   
8        H1       H1       H1       H1       H1            -15.829040   
17       H9       H9       H9       H9       H9            -16.555560   
9        H1       H1       H1       H1       H1            -16.681197   
11       H1       H1       H1       H1       H1            -17.082928   

    StdErrLogProb_Moderate  MeanLogProb_Weak  StdErrLogProb_Weak  \
15                0.372122         -7.884102            0.101984   
1                 0.408493         -7.940729            0.075671   
22                0.558482         -7.931012            0.093875   
21                0.437457     